# Initializations

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

import matplotlib.pyplot as plt

import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder  
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

#Models
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

# Model evaluation und visualisation

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Hyperparameter optimisation
from sklearn.model_selection import GridSearchCV

# to make this notebook's output stable across runs
np.random.seed(842)

# Load and investigate data

In [2]:
# Source: https://www.kaggle.com/utathya/electricity-consumption

energy = pd.read_csv('Electricity_train.csv')

In [3]:
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26496 entries, 0 to 26495
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       26496 non-null  int64  
 1   datetime                 26496 non-null  object 
 2   temperature              26496 non-null  float64
 3   var1                     26496 non-null  float64
 4   pressure                 26496 non-null  float64
 5   windspeed                26496 non-null  float64
 6   var2                     26496 non-null  object 
 7   electricity_consumption  26496 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 1.6+ MB


In [4]:
# Check the categorical data
print(energy['var2'].value_counts() )

A    25239
C     1040
B      217
Name: var2, dtype: int64


# Process the data

The data is a time-series of hourly energy consumptions. This usually calls for a time-series analysis, e.g. with Recurrent Neural Networks. 
Here we take a different approach and try to predict the consumption for any day, simply given the weather data. To be able to account for seasons, we include the energy consumption of the previous day to our prediction. See below for the business case. 

In [5]:
# The previous day can be found by shifting the dataset by 24 places. 
# This creates unknown data (NaN) for the first 24 rows, which are thus deleted
energy['consumption_previous_day'] = energy['electricity_consumption'].shift(24)
energy = energy.dropna()
# droping the first 24 entries did not adapt the index, so we reset it to start again from 1 
energy = energy.reset_index()
energy=energy.drop('index',axis=1) 

In [6]:
# The feature ID is not useful for us
energy=energy.drop('ID',axis=1) 

In [7]:
# we extract the hour of the day, which will have a significant effect on the energy consumption
energy['hour_of_day'] = [int(date_string[11:13]) for date_string in energy['datetime']]
energy.drop('datetime', axis=1, inplace=True)

In [8]:
# Separate features and label 
labels = energy['electricity_consumption']
energy.drop('electricity_consumption', axis=1, inplace=True)

# Test-Train split
We avoid further investigation of the full dataset. Instead, we split the data and work on training data only

In [9]:
# With the category 'var2' being unevenly distributed, we chose to stratify it.
# without knowing the details of this variable, we cannot decide for sure, if this is reasonable

X_train, X_test, train_labels, test_labels = train_test_split(
    energy, labels.values, stratify=energy['var2'], test_size=0.2)

# Reset the indices:
X_test.reset_index(inplace=True, drop=True)
X_train.reset_index(inplace=True, drop=True)

# Pre-process data:

Fill missing values for numerals

Encode categorical features

In [10]:
# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [11]:
# One-Hot-Encoder replaces categorical features by boolean features, stating wether a certain category is true or not
cat_feature_names = ['var2']
cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_feature_names)),
        ('cat_encoder', OrdinalEncoder()),
    ])

# We run the pipeline to check, whether it runs with no problems
temp = cat_pipeline.fit_transform(X_train)

In [12]:
all_attribs = X_train.columns 
num_attribs = [attribute for attribute in all_attribs if attribute not in cat_feature_names]

num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

# We run the pipeline to check, whether it runs with no problems
temp = num_pipeline.fit_transform(X_train)

In [13]:
#Combine both imputers and get cleaned data

all_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

X_prepared = all_pipeline.fit_transform(X_train)

In [14]:
print(X_prepared.shape, train_labels.shape)

(21177, 7) (21177,)


# Regression

In [15]:
# Introducing polynomial features drastically increases the dimension:
poly_features = PolynomialFeatures(degree=3, include_bias=False)  # YL: Best degree
X_poly = poly_features.fit_transform(X_prepared)
print(X_poly.shape)

(21177, 119)


In [16]:
# A Lasso regularisation reduces the number of nonzero coefficients,
#  making the polynomial regression easier to interpret
cubic_regression_lasso = Lasso(alpha=0.01, max_iter=1e5)   # YL: Best Alpha
cubic_regression_lasso.fit(X_poly, train_labels)
score = cubic_regression_lasso.score(X_poly, train_labels)
print(' r^2 %8.3f \n' % score)

 r^2    0.425 



In [17]:
np.set_printoptions(precision=0, suppress=True)
print(cubic_regression_lasso.coef_)

[-65.  55.  -8. -52.  39.  39.   0.  10.  -6.  -5.  47. -27.  -5.  39.
  -5.   9. -18.  32.  -1.   0.  -2.   1.  -1.  -1.   5.  11. -24.  -6.
  93.  -7.   6. -38.  -6.  20.   0.  -2.  -7.   3.  -9.   8.  -2.  -0.
  28. -12.  16. -14.   0.  13.  -4.  -1.   0.   2.  -4.  -4.  13.   2.
  -2.   2.   1.  12.  -8.   4. -12.  -6.   7.   6.   2.  -2.   0.  -1.
  -4.   2.   1.  -0.   4. -16.  -0.  16.  -2.  -0.  -6.   2.   1.  -8.
  -2.  -1.  -2.   3.   1.  -0.   5.   1.  -2.   1.   0.   3.  -1.   1.
   0.  -1.   2.   1.  10.   2.  -2.  21.   3.  -1. -57.   0.  -2.   3.
   0.  -1.  12. -13.   2. -14.  -7.]


# Model evaluation

In [18]:
# Re-Create and the optimal model  # YL
def PolynomialRegression(degree=3, alpha=.01, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), Lasso(alpha, max_iter=2e4, **kwargs))

cubic_regression_lasso = PolynomialRegression()
cubic_regression_lasso.fit(X_prepared, train_labels)

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('lasso', Lasso(alpha=0.01, max_iter=20000.0))])

In [19]:
# Transform the test data using the pipeline defined earlier
X_test_prepared = all_pipeline.transform(X_test)

In [20]:
# predict the test data:
prediction_test = cubic_regression_lasso.predict(X_test_prepared)

In [21]:
score = cubic_regression_lasso.score(X_test_prepared, test_labels)
print(' r^2 %8.3f \n' % score)

 r^2    0.437 



In [22]:
round(mean_squared_error(test_labels, prediction_test))

6679

# Comparison to trivial prediction

Often the error value does not say much about the quality of the model, as it highly depends on the data. One way to assess the quality is by comparing to a trivial prediction. This can be the mean value of a subclass or for time-series a data point of the past. Here, we use the electricity consumption of the previous day as the trivial prediction. 

Note that this is only useful, if we evaluate the model on a daily basis. If we require a prediction for the next year, the consumption of the previous day is clearly not available as an estimate.

In [23]:
labels_previous = X_test['consumption_previous_day'].values

In [24]:
round(mean_squared_error(test_labels, labels_previous))

14670

# Business Case

A good prediction of the electricity consumption is essential for the costing of electricity generation. The costing covers capital cost, operational cost and ramping up cost and time with considerbale differences between technologies and fuels. For example, Coal-fired combustion turbine capital cost is typically 1,00 USD/kW (US dollar per thousand kilo-watt), operation cost is typically 0.04 USD/kWh and typical ramp-up time is an hour. These costs for Natural gas combustion turbine are 800 USD/kW, 0.10 USD/kWh (US dollar per thousand kilo-watt hour) and ramp-up of ten minutes. A typical operational strategy is to operate coal (or oil) turbines to provide the required normal capacity and to use natural gas turbines when there are short-term hikes in required capacity. It usually also makes sense to have large coal or oil turbines where operational costs are low and small gas turnines that can be ramped-up and turned off quickly as more or less un-predicted capacity is required. 

For a detailed course of the economics of electricity see https://www.e-education.psu.edu/ebf483/node/517) 

# Task 1:

When the  prediction of the required capacity is higher than the coal-turbines capacity, the plant will put into operation gas-turbines with higher operational cost. One possible issue is then how many times we predict a higher capacity that does not materialize. Each such occurance will cost the ramp-up of a turbine. Below we count the time this mis-prediction happens (each is per hour) and compare the Laso cubic model to the trivial model of the previous day.

In [25]:
def SumPositiveErrorsOccurance(actual,predicted):
    d1= actual-predicted
    d2=d1>0
    return d2.sum()

In [26]:
np.set_printoptions(precision=0, suppress=True)
print(' Error versus actual      : ',round(SumPositiveErrorsOccurance(test_labels, prediction_test)) )
print(' Error versus previous day: ',round(SumPositiveErrorsOccurance(test_labels, labels_previous)) )

 Error versus actual      :  2301
 Error versus previous day:  2920


# Task 2:

The plant tries to use the minimal number of coal-turbines and no gas-turnines which are costlier. When the  prediction of the required capacity is higher than the coal-turbines capacity, the plant will put into operation gas-turbines with higher operational cost. One possible issue is then how many times we predict a lower capacity that does not materialize. Each such occurance will cost the ramp-up of a turbine. Below we count the times this mis-prediction happens (each is per hour) and compare the Laso cubic model to the trivial model of the previous day.

In [27]:
def SumNegativeErrorsOccurance(actual,predicted):
    d1= actual-predicted
    d2=d1<0
    return abs(d2.sum())

In [28]:
np.set_printoptions(precision=0, suppress=True)
print(' Error versus actual      : ',round(SumNegativeErrorsOccurance(test_labels, prediction_test)) )
print(' Error versus previous day: ',round(SumNegativeErrorsOccurance(test_labels, labels_previous)) )

 Error versus actual      :  2994
 Error versus previous day:  2287


# Task 3:

With the positive and negative occurance errors, it is not clear which model is better. A detailed modelling of the power plant would help -- what is the capacity of the oil and gas turbines, what is the cost of ramp-up, etc. However, operational managers have also their rule of thumb, the negative errors (capacity is missing) are much worse than positive errors (active capacity is idle). The reason is that negative errors require them to buy capacity from other providers and to quickly turn on additional turbines, while positive errors cost some money, but they are not critical to the operation. The following simple code models this rule of thumb: 

In [29]:
factor=10
#
p_model=SumPositiveErrorsOccurance(test_labels, prediction_test)
p_guess=SumPositiveErrorsOccurance(test_labels, labels_previous)
#
n_model=SumNegativeErrorsOccurance(test_labels, prediction_test)
n_guess=SumNegativeErrorsOccurance(test_labels, labels_previous)
#
print(  p_model, p_guess, n_model,n_guess  )
print(  (p_model-p_guess) , (n_model-n_guess)  )
print(  (p_model-p_guess) + (n_model-n_guess)  )
print(  (p_model-p_guess) + factor*(n_model-n_guess)  )

2301 2920 2994 2287
-619 707
88
6451


In [30]:
print(sum(test_labels==prediction_test))
print(sum(test_labels==labels_previous))

0
88


# Task 4:

When gas turbines are required, their operation is costlier, so now we check how the models perform as errors require costlier turbines. This issue is similar to Task 2, but now we count the actual gas-turnine capacity not just an occurance of an error.

In [31]:
def SumNegativeErrors(actual,predicted):
    d1= actual-predicted
    d2=d1[d1<0]
    return abs(d2.sum())

In [32]:
np.set_printoptions(precision=0, suppress=True)
print(' Error versus actual      : ',round(SumNegativeErrors(test_labels, prediction_test)) )
print(' Error versus previous day: ',round(SumNegativeErrors(test_labels, labels_previous)) )

 Error versus actual      :  148230
 Error versus previous day:  213057
